# HW 7: 自駕車間的溝通 (網路工程)

2020.11.13

## Tutorial

- [TA video placehold]()
- [The Internet (11:58)](https://www.youtube.com/watch?v=AEaKrq3SpW8)
- [UDP and TCP: Comparison of Transport Protocols (11:34)](https://www.youtube.com/watch?v=Vdc8TCESIg8)

## Objective

- Learn socket programming to exchange data between different computer
- Understand how computer communicate through network
- Understand difference between two common OSI 4 protocol `TCP` and `UDP`

## Problem

在前面的 Lab 中，我們都是透過 `SSH` 登入進小鴨車  
使用終端機指令或 `python` 程式來控制小鴨車馬達、車燈、相機等  

如果想要提供其他使用者，例如：同學、朋友等 也來控制小鴨車  
就必須提供他們 `SSH` 登入的能力，但是這樣一方面給予權限過多，另一方面使用的門檻也較高

因此問題是否能夠給予其他使用者一個簡單的客戶端 (`client`)  
透過網路給予受限的功能，諸如小鴨車前進、後退等，但無法刪除整個作業系統 `rm -rf /`
讓他們也能體驗奔馳小鴨車的樂趣

## Requirement

利用 `socket` 使用 `TCP` 與 `UDP` 協定提供其他聯網裝置與小鴨車溝通，以控制小鴨車馬達與車燈

> `socket` 使用方法可以參考 [`Reference`](#Reference)

- 在小鴨車上，使用 `TCP` 監聽 `54760` 的埠號 (port)，一直接受一個 byte 的資料去控制小鴨車馬達的行為
  - 只需要處理同時有一個 client 的情況，不用處理同時有多個 client 使用
    - 換句話說，不用使用到 多執行緒 `mutlithreading` 或 多進程 `mutliprocess`
  - 要能夠更換 client，有可能出現聯網裝置 A 連上小鴨車控制後斷開，換聯網裝置 B 連上小鴨車控制
  - 當處理完資料後，要回應一個 `b'\x21'` 告知聯網裝置已收到資料 
  - byte 資料對應的行為如下：

| one byte data | action |
| --- | --- |
| `b'\x20'` | stop |
| `b'\x77'` | move forward |
| `b'\x73'` | move backward |
| `b'\x61'` | turn left |
| `b'\x64'` | turn right |

- 在小鴨車上，使用 `UDP` 綁定 `54760` 的埠號 (port)，一次接受五個 byte 的資料去控制小鴨車車燈的行為
  - 五個 byte 個別對應到 前左燈、前中燈、前右燈、後左燈、後右燈
  - 可能的範例如下
    - `b'\x64\x64\x64\x64\x64'`: 全部燈皆為暗掉，因此都沒有點亮
    - `b'\x72\x72\x72\x62\x62'`: 車前三個燈為紅色，車後兩個燈為藍色
  - 五個 byte 中每個單一的 byte 資料對應的行為如下：

|one byte data|action|
| --- | --- |
| `b'\x77'` | white |
| `b'\x64'` | dark |
| `b'\x72'` | red |
| `b'\x67'` | green |
| `b'\x62'` | blue |

- 完成前述兩項要求後，應可以使用助教提供的程式碼連接上小鴨車並控制  
 
從 `Github` 取得程式碼，這邊有幾個選項可用，選擇適合自己的方式

1. 使用 `git` 克隆程式碼，適合電腦上有安裝 `git` 指令

```bash
$ git clone -b Lab7 https://github.com/GrassLab/NCTU_CS_ZOO.git Lab7
```

2. 透過 `Github` 提供的程式碼下載功能，先選擇到 `Lab7` 的分支，之後點擊 `Download ZIP`

![download zip](https://imgur.com/MmgVSnH.png)

取得程式碼後，使用方法如下

```bash
$ cd Lab7/duckiebot_cs_zoo
$ python -m app.remote_joystick duckie-xxx.local 54760
connect to duckie-xxx.local:54760
pressed w, a, s, d and space as joystick when focus on Window
```

## Notice

- 由於需要與小鴨車溝通，如遇到網路無法連上請在討論區描述問題或私信助教
- 如遭遇網路問題無法實際在小鴨車上操作，可以先在自己電腦熟悉 `TCP` 與 `UDP` 的使用
- 注意在監聽與綁定時，host 參數的使用，例如: `localhost` 與 `0.0.0.0` 的差別
- deadline: 2020.11.20 before class

## Reference

- [Python3.6 官方 socket 參考文件](https://docs.python.org/3.6/library/socket.html)
  - 助教也都用這個，好文件，不用嗎？
- [OSI Model Explained 影片解釋 OSI 7](https://www.youtube.com/watch?v=vv4y_uOneC0)

### 如何建立兩個使用 `TCP` 互相連接的 `socket`

> 底下程式碼是可以直接複製並且執行的

In [1]:
import socket

## create two socket using TCP/IPv4 ##

a = socket.socket(
    family=socket.AF_INET, # mean useing IPv4
    type=socket.SOCK_STREAM, # mean using TCP
)
b = socket.socket(
    family=socket.AF_INET, # mean useing IPv4
    type=socket.SOCK_STREAM, # mean using TCP
)

## first socket `a` as server ##

# select using which host and port
a.bind(('localhost', 0))
# prepare for handling TCP 3-way handshake
a.listen()
# get (host, port)
server_addr: tuple = a.getsockname()

## second socket `b` as client ##

# connect to first socket `a`
b.connect(server_addr)

assert b.getpeername() == a.getsockname()

## get connected socket with `b` ##
c, c_addr = a.accept()

assert c_addr == b.getsockname()
assert c.getsockname() == a.getsockname()
assert c.getpeername() == b.getsockname()

## try to communicate ##

assert b.send(b'\x00') == 1 # to socket `c`
assert c.send(b'\x11') == 1 # to socket `b`
assert b.recv(1) == b'\x11' # from socket `c`
assert c.recv(1) == b'\x00' # from socket `b`

## when disconnect ##

# disconnect
b.close()
assert c.recv(1) == b'' # because socket `b` is closed

# good habit to close unused socket
a.close()
c.close()

### 如何建立兩個使用 `UDP` 交換資料的 `socket`

> 底下程式碼是可以直接複製並且執行的

In [2]:
import socket

## create two socket using UDP/IPv4 ##

a = socket.socket(
    family=socket.AF_INET, # mean useing IPv4
    type=socket.SOCK_DGRAM, # mean using UDP
)
b = socket.socket(
    family=socket.AF_INET, # mean useing IPv4
    type=socket.SOCK_DGRAM, # mean using UDP
)

# select using which host and port
a.bind(('localhost', 0))
b.bind(('localhost', 0))

## try to communicate ##

assert a.sendto(b'\x00', b.getsockname()) == 1 # to socket `b`
assert b.sendto(b'\x11', a.getsockname()) == 1 # to socket `a`
assert a.recvfrom(1) == (b'\x11', b.getsockname()) # from socket `b`
assert b.recvfrom(1) == (b'\x00', a.getsockname()) # from socket `a`

# send to self
assert a.sendto(b'\x66', a.getsockname()) == 1
assert a.recvfrom(1) == (b'\x66', a.getsockname())

### 如何真的跨越網路到另一端

上面的範例都只有在同一台聯網裝置執行，但作業的 `Requirement` 是要在小鴨車與另一台聯網裝置間  
可以參考 [官方範例](https://docs.python.org/3.6/library/socket.html#example)，看第一個即可

要注意 `host` 在 `'localhost'` 與 `'0.0.0.0'` (或範例中 `''`) 兩者間區別與使用